In [18]:
from typing import TypedDict, List
from dotenv import load_dotenv
from langgraph.graph import StateGraph, START, END
from langchain.chat_models import init_chat_model
from pydantic import BaseModel

load_dotenv()

llm = init_chat_model("openai:gpt-4o-mini")

In [19]:
class State(TypedDict):
    dish: str
    ingredients: list[dict]
    recipe_steps: str
    plating_instructions: str


class Ingredient(BaseModel):
    name: str
    quitantity: str
    unit: str


class IngredientsOutput(BaseModel):
    ingredients: List[Ingredient]

In [20]:
def list_ingredients(state: State):
    structured_llm = llm.with_structured_output(IngredientsOutput)
    response = structured_llm.invoke(
        f"{state['dish']} 음식을 만들기 위한 5개에서 8개의 재료 이름을 나열해 주세요."
    )
    return {"ingredients": response.ingredients}


def create_recipe(state: State):
    response = llm.invoke(
        f"{state['dish']} 요리를 만들기 위한 단계별 조리법을 작성해 주세요. 재료는 다음과 같습니다: {state['ingredients']}"
    )
    return {"recipe_steps": response.content}


def describe_plating(state: State):
    response = llm.invoke(
        f"{state['dish']}를 멋지게 플레이팅하는 방법 다음 레시피를 기반으로 설명해 주세요: {state['recipe_steps']}"
    )
    return {"plating_instructions": response.content}

In [21]:
graph_builder = StateGraph(State)

graph_builder.add_node("list_ingredients", list_ingredients)
graph_builder.add_node("create_recipe", create_recipe)
graph_builder.add_node("describe_plating", describe_plating)

graph_builder.add_edge(START, "list_ingredients")
graph_builder.add_edge("list_ingredients", "create_recipe")
graph_builder.add_edge("create_recipe", "describe_plating")
graph_builder.add_edge("describe_plating", END)

graph = graph_builder.compile()

In [22]:
graph.invoke({"dish": "hummus"})

{'dish': 'hummus',
 'ingredients': [Ingredient(name='병아리콩', quitantity='1', unit='컵'),
  Ingredient(name='타히니', quitantity='1/4', unit='컵'),
  Ingredient(name='올리브 오일', quitantity='3', unit='테이블스푼'),
  Ingredient(name='레몬 주스', quitantity='3', unit='테이블스푼'),
  Ingredient(name='다진 마늘', quitantity='2', unit='쪽'),
  Ingredient(name='소금', quitantity='1/2', unit='티스푼'),
  Ingredient(name='물', quitantity='2-3', unit='테이블스푼'),
  Ingredient(name='큐민 (선택사항)', quitantity='1/2', unit='티스푼')],
 'recipe_steps': '허무스(hummus) 만드는 방법을 단계별로 설명드리겠습니다. 아래 조리법을 따라 주시면 맛있는 허무스를 만들 수 있습니다!\n\n### 필요 재료\n- 병아리콩: 1컵\n- 타히니: 1/4컵\n- 올리브 오일: 3 테이블스푼\n- 레몬 주스: 3 테이블스푼\n- 다진 마늘: 2쪽\n- 소금: 1/2 티스푼\n- 물: 2-3 테이블스푼\n- 큐민 (선택사항): 1/2 티스푼\n\n### 조리 단계\n\n1. **병아리콩 준비하기**  \n   - 병아리콩을 하룻밤 동안 물에 담가서 불려줍니다.  \n   - 다음 날, 불린 병아리콩을 체에 받쳐 물기를 제거합니다.\n\n2. **병아리콩 삶기**  \n   - 큰 냄비에 물을 채우고 병아리콩을 넣어 끓입니다.  \n   - 끓기 시작하면 불을 줄이고 중하火로 45분에서 1시간 정도 삶아줍니다.  \n   - 병아리콩이 부드러워지면 체에 받쳐 물기를 제거한 후, 식힙니다.\n\n3. **믹서에 재료 넣기**  \n   - 믹서기